In [1]:
import time
from PIL import ImageGrab, ImageEnhance, ImageOps
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [2]:
from winsound import Beep
def ring_alarm():
    Beep(300, 100)
    Beep(600, 500)
    Beep(500, 100)

In [3]:
import win32api, win32con
def clickRefresh():
    x, y = 220, 165
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,x,y,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,x,y,0,0)
    
    time.sleep(0.2)

In [4]:
wishList = {
    "Watchful Arbelist's Bracers" : 10000000,
    "Legionnaire’s Bloodstained Sabatons" : 500000,
    "Soldier's Stoneband Wristguards" : 500000,
    "Supple Supplicant's Gloves" : 500000,
    "Fallen Templar's Gauntlets" : 500000,
    "Acolyte’s Velvet Bindings" : 600000,
    "Barkweave Wristwraps" : 500000,
    "Decadent Nathrian Shawl" : 500000,
    "Stud-Scarred Footwear" : 500000,
    "Relic of the First Ones" : 200000,
    "Gast's Cloak of Compression" : 200000,
    "Punishing Loop" : 500000,
    "Elohin’s Burden of Patience" : 200000,
    "Azeharys' Lunar Vell" : 200000,
    "Gaiazelle's Spiked Collar" : 200000
           }

In [5]:
from cfuzzyset import cFuzzySet as FuzzySet

# Loads all possible card names to match
def init_fuzzy_matcher(items):
    fuzzyMatcher = FuzzySet()

    for key in list(items.keys()):
        fuzzyMatcher.add(key)
        
    return fuzzyMatcher

def get_fuzzy_best_match(fuzzyMatcher, cardName, debug = False):
    match = fuzzyMatcher.get(cardName)
    if match:
        bestMatch = match[0]
        if bestMatch[0] > 0.25:
            if debug:
                print(cardName, '=>', bestMatch, 'found')
            return bestMatch[1]
    return None 

fuzzyMatcher = init_fuzzy_matcher(wishList)

In [6]:
def imgPrep(img):
    yield img
     
    newImg = img.point(lambda p: p > 69 and 255)
    newImg = ImageEnhance.Color(newImg).enhance(0)
    yield newImg
    
    for thresh in range(40, 90, 2):
        fn = lambda x : 255 if x > thresh else 0
        newImg = img.convert('L').point(fn, mode='1')
        yield newImg

In [7]:
def getAHimg():
    img = ImageGrab.grab()
    img = img.crop((215, 215, 695, 615))
    return img

In [8]:
def comparePriceToWishlist(name, price, debug = False):
    if debug:
        print(name, price)
        
    if wishList[name] >= int(price):
        print('Go buy {} !!!'.format(name))
        ring_alarm()

In [9]:
nb_obj = 20

def extract_info(img, debug = False):
    width, height = img.size
    item_height = int(height/nb_obj)

    for offset in range(0, height, item_height):
        nameImg = img.crop((170, offset, width, offset + item_height))
        priceImg = ImageOps.invert(img.crop((0, offset, 66, offset + item_height)))

        if debug:
            display(nameImg)
            display(priceImg)

        custom_config = r'--oem 1 --psm 6'
        for cleanedNameImg in imgPrep(nameImg):
            imgdata = pytesseract.image_to_data(cleanedNameImg, config=custom_config, lang="eng", output_type=pytesseract.Output.DICT)
            name = ' '.join([x for i,x in enumerate(imgdata['text']) if int(imgdata['conf'][i]) >= 30])
            if len(name) > 20:
                name_match = get_fuzzy_best_match(fuzzyMatcher, name)
                if name_match:


                    custom_config = r'--oem 3 --psm 10 -c tessedit_char_whitelist=0123456789'
                    for cleanedPriceImg in imgPrep(priceImg):
                        imgdata = pytesseract.image_to_data(cleanedPriceImg, lang='eng', config=custom_config, output_type=pytesseract.Output.DICT)
                        price = ' '.join([x for i,x in enumerate(imgdata['text']) if int(imgdata['conf'][i]) >= 0])
                        if len(price) > 3:
                            break


                    comparePriceToWishlist(name_match, price)
                    if debug:
                        print(name_match, price)

In [10]:
while True:
    clickRefresh()
    img = getAHimg()
    extract_info(img, debug = True)
    time.sleep(20)

Legionnaire’s Bloodstained Sabatons 2499990


Fallen Templar's Gauntlets 1900000


Soldier's Stoneband Wristguards 1500000


Barkweave Wristwraps 1200000


Go buy Watchful Arbelist's Bracers !!!
Watchful Arbelist's Bracers 1200000


Supple Supplicant's Gloves 1200000


Gaiazelle's Spiked Collar 1000000


Decadent Nathrian Shawl 700000


KeyboardInterrupt: 